# UAV Network Analysis - Main Notebook

This notebook provides a clean, organized workflow for analyzing UAV design networks.
All complex logic is encapsulated in modules.

## Analysis Pipeline:
1. **Setup** - Initialize environment and load utilities
2. **Data Loading** - Load all design data from JSON files  
3. **CAD Visualization** - Plot 3D CAD models
4. **Geometry Parsing** - Extract geometric information from design trees
5. **Position Calculation** - Calculate realistic 3D positions
6. **Network Visualization** - Plot network graphs in 3D
7. **Complexity Analysis** - Calculate Shannon entropy-based complexity metrics
8. **Advanced Visualizations** - Create detailed plots and comparisons

---

## Block 1: Setup and Initialization

In [ ]:
# Import modules
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

# Add modules to path
sys.path.insert(0, str(Path.cwd() / 'modules'))

# Import our custom modules
from modules import config, setup, plot_utils
from modules import data_loader, cad_plotter, geometry_parser
from modules import position_calculator, network_plotter, complexity_analyzer
from modules import design_selector, complexity_radar_plot, complexity_box_violin_plot

# Initialize environment
env_state = setup.initialize_environment()

# Configure plot style
plot_utils.setup_plot_style()

# Print configuration
config.print_config()

## Block 2: Load All Design Data

Load design data from JSON files and build network graphs.

In [ ]:
# Get sorted design directories
design_dirs = setup.get_sorted_designs(config.DATA_ROOT)

# Load all design data
all_designs_data = data_loader.load_all_designs(design_dirs, verbose=True)

# Print statistics
data_loader.print_design_stats(all_designs_data)

## Block 3: Plot CAD Models

Visualize 3D CAD models from STL files.

In [ ]:
# Plot all CAD models in a grid
fig_cad = cad_plotter.plot_all_cad_models(design_dirs, n_cols=5, save=True)

## Block 4: Verification - Component Extraction

Verify that component extraction is working correctly.

In [ ]:
print("\n" + "=" * 80)
print("VERIFICATION - COMPONENT EXTRACTION ACCURACY")
print("=" * 80)

# Check first few designs
for design_name in sorted(all_designs_data.keys(), key=setup.get_design_number)[:3]:
    data = all_designs_data[design_name]
    print(f"\n📋 {design_name}:")
    print(f"   Design name: {data['payload']['name']}")
    print(f"   Components: {data['num_nodes']}")
    print(f"   Connections: {data['num_edges']}")
    
    # Count component types
    from collections import Counter
    comp_types = [attrs.get('component_type') for node, attrs in data['G'].nodes(data=True)]
    type_counts = Counter(comp_types)
    print(f"   Component types: {len(type_counts)}")
    for comp_type, count in type_counts.most_common(5):
        print(f"      {comp_type:20s} {count}")

## Block 5: Parse Design Tree for Geometry

Extract geometric information (hub type, arm length, etc.) from design trees.

In [ ]:
# Parse geometry for all designs
designs_geometry = geometry_parser.parse_all_geometries(all_designs_data, verbose=True)

# Print geometry summary
geometry_parser.print_geometry_summary(designs_geometry)

## Block 6: Calculate Realistic 3D Positions

Calculate node positions that mimic actual CAD placement.

In [ ]:
# Calculate 3D positions for all designs
all_positions_3d = position_calculator.calculate_all_positions(
    all_designs_data, 
    designs_geometry, 
    verbose=True
)

## Block 7: Plot 3D Network Graphs

Visualize network graphs in 3D for all designs.

In [ ]:
# Plot all network graphs in 3D (3 rows x 5 columns for 15 designs)
# Note: Complexity levels will be determined automatically
# Design 1 = Least Complexity, Design 14 = Medium Complexity
# Design 5 = Highest Complexity, Design 12 = Non-Classical
fig_networks = network_plotter.plot_all_networks_3d(
    all_designs_data,
    all_positions_3d,
    n_cols=5,  # 5 columns for 3x5 grid (15 designs)
    save=True,
    complexity_results=None  # Will use node count to determine complexity if not provided
)
plt.show()  # Display the plot

## Block 8: Plot Selected Designs (Representative Sample)

Select and plot a few representative designs for detailed analysis.

In [ ]:
# Select 4 representative designs based on PDF specifications:
# Design 1: Least Complexity
# Design 14: Medium Complexity
# Design 5: Highest Complexity
# Design 12: Non-Classical Complexity
selected_designs = design_selector.select_representative_designs(all_designs_data)
design_selector.print_selected_designs(selected_designs)

# Get design names for plotting
selected_names = [stats['design_name'] for label, stats in selected_designs]
selected_indices = []
design_names = sorted(all_designs_data.keys(), key=setup.get_design_number)
for name in selected_names:
    if name in design_names:
        selected_indices.append(design_names.index(name))

# Plot selected networks
fig_selected = network_plotter.plot_selected_networks_3d(
    all_designs_data,
    all_positions_3d,
    selected_indices,
    titles=selected_names,
    save=True
)
plt.show()  # Ensure plot is displayed

## Block 9: Complexity Analysis Using Shannon Entropy

Calculate structural complexity metrics based on Shannon entropy.
Reference: Sinha & de Weck (2020)

In [ ]:
# Analyze complexity for all designs
complexity_results = complexity_analyzer.analyze_design_complexity(
    all_designs_data,
    verbose=True
)

# Get complexity summary
complexity_summary = complexity_analyzer.get_complexity_summary(complexity_results)

print("\n" + "=" * 80)
print("COMPLEXITY SUMMARY ACROSS ALL DESIGNS")
print("=" * 80)
print(f"\nTotal Complexity:")
print(f"  Mean: {complexity_summary['total_complexity']['mean']:.3f} ± {complexity_summary['total_complexity']['std']:.3f}")
print(f"  Range: [{complexity_summary['total_complexity']['min']:.3f}, {complexity_summary['total_complexity']['max']:.3f}]")
print(f"\nDiversity (H_div):")
print(f"  Mean: {complexity_summary['diversity']['mean']:.3f} ± {complexity_summary['diversity']['std']:.3f}")
print(f"\nFlexibility (H_flex):")
print(f"  Mean: {complexity_summary['flexibility']['mean']:.3f} ± {complexity_summary['flexibility']['std']:.3f}")
print(f"\nCombinability (H_comb):")
print(f"  Mean: {complexity_summary['combinability']['mean']:.3f} ± {complexity_summary['combinability']['std']:.3f}")
print("=" * 80)

## Block 10: Complexity Comparison Plot

Create a comparison plot of complexity metrics across designs.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract complexity data
design_names = sorted(complexity_results.keys(), key=setup.get_design_number)
total_complexities = [complexity_results[d]['system_entropies']['total_complexity'] for d in design_names]
diversities = [complexity_results[d]['system_entropies']['H_diversity'] for d in design_names]
flexibilities = [complexity_results[d]['system_entropies']['H_flexibility'] for d in design_names]
combinabilities = [complexity_results[d]['system_entropies']['H_combinability'] for d in design_names]

# Create comparison plot
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Total complexity
axes[0, 0].bar(range(len(design_names)), total_complexities, color='steelblue')
axes[0, 0].set_title('Total System Complexity', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Design Index')
axes[0, 0].set_ylabel('Complexity')
axes[0, 0].grid(True, alpha=0.3)

# Diversity
axes[0, 1].bar(range(len(design_names)), diversities, color='coral')
axes[0, 1].set_title('Component Diversity (H_div)', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Design Index')
axes[0, 1].set_ylabel('Entropy')
axes[0, 1].grid(True, alpha=0.3)

# Flexibility
axes[1, 0].bar(range(len(design_names)), flexibilities, color='lightgreen')
axes[1, 0].set_title('Connection Flexibility (H_flex)', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Design Index')
axes[1, 0].set_ylabel('Entropy')
axes[1, 0].grid(True, alpha=0.3)

# Combinability
axes[1, 1].bar(range(len(design_names)), combinabilities, color='plum')
axes[1, 1].set_title('Structural Combinability (H_comb)', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Design Index')
axes[1, 1].set_ylabel('Entropy')
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Complexity Metrics Comparison - All Designs', fontsize=16, fontweight='bold')
plt.tight_layout()

# Save and show
plot_utils.save_and_show(fig, 'Complexity_Comparison_All_Designs')
plt.show()

## Block 11: Node Complexity Analysis for Selected Designs

Calculate node-level complexity for the 4 representative designs.

In [ ]:
import numpy as np

# Calculate node complexity for selected designs
# Prepare selected_designs with node complexity data
selected_designs_with_complexity = []
for label, stats in selected_designs:
    design_name = stats['design_name']
    if design_name in complexity_results:
        node_complexities = complexity_results[design_name]['node_complexities']
        # Calculate cumulative complexity
        cumulative = sum([nc.get('total_complexity', 0.0) for nc in node_complexities.values()])
        
        selected_designs_with_complexity.append((
            label,
            {
                **stats,
                'node_complexity': node_complexities,
                'cumulative_complexity': cumulative
            }
        ))

print("\n" + "=" * 80)
print("NODE COMPLEXITY FOR SELECTED DESIGNS")
print("=" * 80)

for label, stats in selected_designs_with_complexity:
    design_name = stats['design_name']
    node_comp = stats.get('node_complexity', {})
    cum_comp = stats.get('cumulative_complexity', 0)
    
    if node_comp:
        complexities = [nc.get('total_complexity', 0.0) for nc in node_comp.values()]
        print(f"\n{label} ({design_name}):")
        print(f"  Cumulative: {cum_comp:.3f}")
        print(f"  Node Complexity Range: {min(complexities):.3f} - {max(complexities):.3f}")
        print(f"  Mean Node Complexity: {np.mean(complexities):.3f}")
        print(f"  Median Node Complexity: {np.median(complexities):.3f}")
        print(f"  Std Deviation: {np.std(complexities):.3f}")

print("\n" + "=" * 80)

## Block 12: Box and Violin Plots for Complexity Distribution

Create box plots and violin plots showing the distribution of node complexity for the 4 selected designs.

In [ ]:
# Create box and violin plots for selected designs
figures_box_violin = complexity_box_violin_plot.create_all_box_violin_plots(
    complexity_results,
    selected_designs=selected_designs_with_complexity,
    save=True,
    verbose=True
)

# All plots are automatically displayed via save_and_show with show=True

## Block 13: Radar Plots for Complexity Metrics

Create radar/spider plots showing multi-dimensional complexity metrics (Flexibility, Combinability, Diversity) for Motor components across all designs.

In [ ]:
# Create radar plots for Motor complexity
sorted_design_names = sorted(complexity_results.keys(), key=setup.get_design_number)

figures_radar = complexity_radar_plot.create_all_radar_plots(
    complexity_results,
    sorted_design_names,
    component_type='Motor',
    save=True,
    verbose=True
)

# All plots are automatically displayed via save_and_show with show=True

## Summary and Next Steps

This notebook provides a complete workflow for UAV network analysis. The modular structure makes it easy to:

1. **Add new analyses** - Create new modules for specialized analysis
2. **Extend visualizations** - Add interactive plots using plotly
3. **Compare designs** - Analyze differences and similarities
4. **Export results** - Save data and plots for publications

### Selected Representative Designs:
- **Design 1**: Least Complexity (29 nodes, 64 edges)
- **Design 14**: Medium Complexity (37 nodes, 84 edges)
- **Design 5**: Highest Complexity (93 nodes, 212 edges)
- **Design 12**: Non-Classical Complexity (78 nodes, 176 edges)

All plots are automatically saved to the `plots/` directory in high resolution (600 DPI) in both PNG and PDF formats, and are displayed in the notebook for immediate viewing.